In [1]:
%autosave 500

Autosaving every 500 seconds


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random as rand
from numpy import array
import itertools
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import zero_one_loss, log_loss
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Open a Dataset

In [4]:
def openDataset(filename):
    dataFile = open(filename, 'r')
    dataFrame = pd.read_csv(dataFile, sep=',', header=0)
    dataFile.close()

    return dataFrame

In [5]:
breast = openDataset('breast-cancer.csv')
breast[' deg-malig'] = breast[' deg-malig'].astype(str)
originalOneHot = pd.get_dummies(breast)

In [6]:
#Decide how much '?' randomly put in the currently row & put it in a random column position; do it for every sample

In [7]:
maxMissingValues = rand.randint (0, 3)

def randomicAssignment ():
    for i in range(0, len(breast)):
        for j in range(0, maxMissingValues):
            missingValues = rand.randint (0, len(breast.columns) - 1)
            breast.iloc[i, missingValues] = '?'
    
    return breast

In [8]:
newBreast = randomicAssignment()
#newBreast

In [9]:
#One-Hot encoding on the dataset with '?'

In [10]:
encodedDF = pd.get_dummies(newBreast)
encodedDF

,class_?,class_no-recurrence-events,class_recurrence-events,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,age_?,...,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_?,irradiat_no,irradiat_yes
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
4,0,1,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0
5,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
6,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
7,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
9,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0


In [11]:
#Divide input & output in with the latter is equal to labels

In [12]:
def preprocessing(dataframe, dataframeLabels):
    y = dataframeLabels.values
    x = dataframe.drop(dataframeLabels.columns, axis=1).values
    
    return x,y

In [13]:
#Split the TrainSet e TestSet, if there is a '?' it is in the TestSet else in the TrainSet

In [14]:
def splitTrainAndTest(x, y, yOriginal, index):
    xTrain = []
    xTest = []
    yTrain = []
    yTest = []
    yTestOriginal = []
    
    for i in range (0, len(y)):
        for j in index:
            if y[i][j] == 1:
                yTest.append(y[i].tolist())
                yTestOriginal.append(yOriginal[i].tolist())
                xTest.append(x[i].tolist())

            else:
                yTrain.append(y[i].tolist())
                xTrain.append(x[i].tolist())
    
    return array(xTrain), array(xTest), array(yTrain), array(yTest), array(yTestOriginal)

In [15]:
#Create the Neural Network

In [16]:
def NeuralNets(xTrain, yTrain, hidden, iterations):
    classifier = MLPClassifier(hidden_layer_sizes = hidden, max_iter = iterations)
    classifier.fit(xTrain, yTrain) 
    
    return classifier

In [17]:
#Create every possible subset, for creating a Neural Network for every of it

In [18]:
def possibleSubsets(df, maximum):
    subsetss = []
    
    for i in range (1, maximum):
        for subset in itertools.combinations(df.columns, i):
            subsetss.append(list(subset))
    
    return subsetss

In [19]:
#Labels is equal to a set of output columns of oneHotEncoding are under consideration

In [20]:
def createLabels(subset, oneHotLabels):
    labels = []
    
    for label in subset:
        for labelOneHot in oneHotLabels:
            if label in labelOneHot:
                labels.append(labelOneHot)
    
    return labels

In [21]:
#Unknown is equal to the list of columns with '?' in oneHotEncoding

In [22]:
def createUnknownLabels(labels, unknown):
    for label in labels:
        if '?' in label:
            unknown.append(label)
    
    return unknown

In [23]:
#Train and predict probabilistic vector

In [24]:
def trainAndPredictProba(classifier, xTest):
    yPredProb = classifier.predict_proba(xTest)
    
    
    return yPredProb

In [25]:
#Common training and prediction

In [26]:
def trainAndPredict(classifier, xTest):
    yPred = classifier.predict(xTest)    

    return yPred

In [27]:
def correctlyReindex(index, yPred):
    k = False
    
    for i in index:
        if k:
            i = i-1
        yPred = np.delete(yPred, i, axis=1)
        k = True
            
    return yPred

In [28]:
def printZeroOneLoss(yTestOriginal, yPred):
    Loss = zero_one_loss(yTestOriginal, yPred)
    
    print("0/1 Loss:")
    print(Loss)
    print("-------------------------------------------")

In [29]:
def printProbabilisticLoss(yTestOriginal, yPred):
    Loss = log_loss(yTestOriginal, yPred)
    
    print("Log Loss:")
    print(Loss)
    print("-------------------------------------------")

In [30]:
def printCase(subset):
    
    print("Case: {subset} with ?".format(subset=subset))
    print("-------------------------------------------")

In [31]:
def printTrainTest(xTrain, xTest):
    
    print("Train-Test sizes:")    
    print("{train}% train".format(train=int(round(100*len(xTrain)/(len(xTrain)+len(xTest))))))
    print("{test}% test".format(test=int(round(100*len(xTest)/(len(xTrain)+len(xTest))))))
    print("-------------------------------------------")

In [32]:
def printLabels(labels):
    
    print("Possible values to predict:")
    print(labels)
    print("--------------------------------------------")

In [33]:
def printProbabilityVector(yPredProb):

    print('Prediction probability for each entry in test:')
    print(yPredProb)
    print("--------------------------------------------")

In [34]:
def printQuestionMarks(newBreast):
    
    for i in range(0, len(newBreast)):
        for j in range(0, len(newBreast.columns) -1):
            if '?' in j:
                print('here')
            
    print('Question marks percentage:')
    
    print("--------------------------------------------")

In [35]:
def printEverything(subset, xTrain, xTest, labels, yPredProb, yTestOriginal, yPred, newBreast):
    
    printCase(subset)
    printTrainTest(xTrain, xTest)
    printLabels(labels)
    printProbabilityVector(yPredProb)
    printZeroOneLoss(yTestOriginal, yPred)
    printProbabilisticLoss(yTestOriginal, yPred)
    #printQuestionMarks(newBreast)

In [36]:
#unknown = lista di colonne con il '?' di oneHotEncoding
#index = indici colonne di unknown
#labels = insieme delle colonne di output che stiamo considerando di oneHotEncoding
#oneHotLabels = colonne del df codificato ('encodedDF')

In [40]:
def probPredict():
    #For every subset of possible columns as output
    for subset in possibleSubsets(breast, 3):
        labels = [] 
        unknown = [] 
        oneHotLabels = encodedDF.columns 
        
        labels = createLabels(subset, oneHotLabels)
        labels_original = [x for x in labels if "?" not in x ]

        #Divide input and output where outputs are equal to output columns in encodedDF
        x, y = preprocessing(encodedDF, encodedDF[labels])
        
        #Equal, but for the original dataset
        xOriginal, yOriginal = preprocessing(originalOneHot, originalOneHot[labels_original])
            
        #Create unknown list
        unknown = createUnknownLabels(labels, unknown)
        
        #create index
        index = [labels.index(x) for x in unknown] # indici colonne con ? con one_hot_encoding
        
        #Split train and test where test is equal yo rows containing '?' in output rows
        xTrain, xTest, yTrain, yTest, yTestOriginal = splitTrainAndTest(x, y, yOriginal, index)     
        
        #Train e predict using Neural Nets
        m = int(len(breast.columns)+len(labels)/2)
        classifier = NeuralNets(xTrain, yTrain, (m*3,m*2), 5000)
        
        yPredProb = trainAndPredictProba(classifier, xTest)
        yPred = trainAndPredict(classifier, xTest)
        
        #Perform the reindex since deleting the considered column it changes
        yPred = correctlyReindex(index, yPred)
        
        printEverything(subset, xTrain, xTest, labels, yPredProb, yTestOriginal, yPred, newBreast)
        print( )
        print( )
        print( )

In [42]:
probPredict()

Case: ['class'] with ?
-------------------------------------------
Train-Test sizes:
74% train
26% test
-------------------------------------------
Possible values to predict:
['class_?', 'class_no-recurrence-events', 'class_recurrence-events']
--------------------------------------------
Prediction probability for each entry in test:
[[1.58117947e-03 7.37609041e-01 2.11303565e-01]
 [3.09563142e-03 2.84754219e-01 7.86737299e-01]
 [2.59962862e-04 9.60399358e-01 2.20914096e-02]
 [5.18409476e-04 8.37031639e-01 2.09305422e-01]
 [1.35820871e-03 8.14399586e-01 1.72760284e-01]
 [8.80709826e-04 7.13971319e-01 1.96709186e-01]
 [1.48031057e-03 8.32241395e-01 1.39357799e-01]
 [6.98215527e-04 3.42711668e-01 6.44396939e-01]
 [3.09762834e-04 8.39291749e-01 1.84826459e-01]
 [5.36736757e-04 7.56778249e-01 2.17434495e-01]
 [2.37832859e-03 9.71032834e-01 3.72343572e-02]
 [2.70648007e-03 7.59332792e-01 2.53168754e-01]
 [9.21432158e-04 9.19330468e-01 7.83402876e-02]
 [1.88701427e-03 9.60033745e-01 4.01017

Case: [' menopause'] with ?
-------------------------------------------
Train-Test sizes:
76% train
24% test
-------------------------------------------
Possible values to predict:
[' menopause_?', ' menopause_ge40', ' menopause_lt40', ' menopause_premeno']
--------------------------------------------
Prediction probability for each entry in test:
[[1.59755511e-03 9.82371311e-01 1.88247613e-02 1.31750130e-02]
 [1.92247589e-04 7.27663148e-03 3.25837192e-04 9.93863911e-01]
 [3.21094228e-03 9.11802950e-01 6.66795995e-02 4.43636403e-02]
 [1.75320502e-03 8.25414713e-01 1.34886230e-02 1.55822428e-01]
 [1.38401005e-03 8.18279878e-01 3.21372602e-02 1.64285848e-01]
 [6.50465773e-04 6.43579124e-01 1.54655766e-02 2.63423085e-01]
 [3.60810940e-04 1.72377549e-01 3.55063376e-03 8.31373532e-01]
 [1.47686225e-03 9.32283135e-01 7.84425519e-02 2.09199108e-02]
 [4.67967745e-04 3.90273720e-01 1.84263815e-03 6.80558254e-01]
 [9.24334931e-04 7.12185084e-01 2.62270192e-02 2.04415193e-01]
 [1.07752752e-03 8.6

Case: [' inv-nodes'] with ?
-------------------------------------------
Train-Test sizes:
73% train
27% test
-------------------------------------------
Possible values to predict:
[' inv-nodes_0-2', ' inv-nodes_12-14', ' inv-nodes_15-17', ' inv-nodes_24-26', ' inv-nodes_3-5', ' inv-nodes_6-8', ' inv-nodes_9-11', ' inv-nodes_?']
--------------------------------------------
Prediction probability for each entry in test:
[[8.64965173e-01 1.98091297e-03 1.40913177e-02 3.52517037e-03
  6.18174010e-02 2.95067792e-02 2.33419841e-02 2.97169191e-03]
 [8.93673307e-01 2.75267753e-03 2.75238018e-02 4.19584765e-03
  3.67367657e-02 1.29035895e-01 4.09376782e-03 1.25314072e-03]
 [7.05247068e-01 1.15702302e-02 3.74461636e-02 1.82900541e-02
  9.09024392e-02 1.54597713e-01 2.63924225e-02 8.97382611e-03]
 [9.11553268e-01 1.15903771e-03 5.01820254e-03 1.24149720e-03
  6.34167474e-02 1.91387702e-02 6.91105067e-03 7.72508236e-04]
 [9.85334821e-01 1.15068645e-03 3.73907719e-03 1.43370680e-03
  1.29043333e-0

Case: [' node-caps'] with ?
-------------------------------------------
Train-Test sizes:
68% train
32% test
-------------------------------------------
Possible values to predict:
[' node-caps_?', ' node-caps_no', ' node-caps_yes']
--------------------------------------------
Prediction probability for each entry in test:
[[8.96024009e-06 9.99996017e-01 1.97547443e-06]
 [1.89762689e-05 3.30722503e-01 8.03320386e-01]
 [3.27351554e-05 9.99943031e-01 2.67328570e-05]
 [6.44847049e-05 9.99869230e-01 1.03077568e-04]
 [2.77811511e-06 9.99999629e-01 1.58222815e-07]
 [1.33644033e-04 9.96675342e-01 3.02826683e-03]
 [2.19681260e-05 9.99995457e-01 2.36851409e-06]
 [1.38119854e-06 9.99999758e-01 5.24010547e-08]
 [4.36478640e-05 9.99903989e-01 6.48936430e-05]
 [3.95329923e-05 9.98501330e-01 7.60900090e-04]
 [4.45400452e-05 9.99954312e-01 3.11461063e-05]
 [3.85191921e-05 9.99999363e-01 2.18509129e-07]
 [5.55179342e-06 9.99999715e-01 9.26452716e-08]
 [5.07481737e-05 9.97423814e-01 1.85551548e-03]
 [6

Case: [' breast'] with ?
-------------------------------------------
Train-Test sizes:
73% train
27% test
-------------------------------------------
Possible values to predict:
[' breast_?', ' breast_left', ' breast_right', ' breast-quad_?', ' breast-quad_central', ' breast-quad_left_low', ' breast-quad_left_up', ' breast-quad_right_low', ' breast-quad_right_up']
--------------------------------------------
Prediction probability for each entry in test:
[[0.22217823 0.33920539 0.17078194 ... 0.0704043  0.05202215 0.0028285 ]
 [0.10997309 0.74669513 0.06090597 ... 0.29099413 0.1214331  0.01677662]
 [0.31365337 0.08760259 0.74220855 ... 0.03602758 0.01366271 0.01263162]
 ...
 [0.28712352 0.02887722 0.72475632 ... 0.27658428 0.25605215 0.00184621]
 [0.56629153 0.6020013  0.00792302 ... 0.0031769  0.0134789  0.0012797 ]
 [0.55280509 0.31497227 0.11445584 ... 0.23309716 0.00486597 0.0687392 ]]
--------------------------------------------
0/1 Loss:
0.9490445859872612
-----------------------

...

Case: [' age', ' irradiat'] with ?
-------------------------------------------
Train-Test sizes:
73% train
27% test
-------------------------------------------
Possible values to predict:
[' age_20-29', ' age_30-39', ' age_40-49', ' age_50-59', ' age_60-69', ' age_70-79', ' age_?', ' irradiat_?', ' irradiat_no', ' irradiat_yes']
--------------------------------------------
Prediction probability for each entry in test:
[[3.44928730e-05 3.50808974e-02 4.67362074e-01 ... 4.56631872e-01
  6.21953036e-01 1.36261612e-02]
 [3.54597025e-05 2.28026788e-02 8.87250159e-01 ... 9.10769130e-01
  1.79513008e-02 1.08853242e-01]
 [2.08126629e-03 8.26483565e-02 2.83317589e-01 ... 3.58297597e-01
  3.18650761e-01 7.98985006e-02]
 ...
 [5.92255256e-07 2.93410344e-01 3.23061841e-03 ... 9.52249085e-01
  1.95668910e-02 5.05508263e-03]
 [3.58701949e-07 3.99088682e-02 6.59429542e-03 ... 9.00177676e-01
  8.60197306e-02 1.40888378e-03]
 [2.51977059e-05 1.02850022e-01 1.15313685e-03 ... 4.71119326e-01
  1.3638274

Case: [' menopause', ' irradiat'] with ?
-------------------------------------------
Train-Test sizes:
74% train
26% test
-------------------------------------------
Possible values to predict:
[' menopause_?', ' menopause_ge40', ' menopause_lt40', ' menopause_premeno', ' irradiat_?', ' irradiat_no', ' irradiat_yes']
--------------------------------------------
Prediction probability for each entry in test:
[[9.73468862e-04 7.76557332e-08 1.92259519e-03 ... 9.98081295e-01
  3.15499423e-03 9.94700533e-05]
 [9.49286563e-01 3.06330254e-02 3.42310746e-04 ... 5.10241187e-07
  9.99998403e-01 7.00886026e-07]
 [9.70565630e-01 1.67996930e-02 2.95871386e-06 ... 2.03463721e-02
  9.99288320e-01 1.09296641e-07]
 ...
 [3.63450215e-03 1.69034971e-01 2.25443627e-01 ... 9.78112440e-01
  1.51384797e-03 4.80522076e-05]
 [8.48665536e-01 2.77757870e-01 4.48453530e-04 ... 1.16528704e-01
  7.07399003e-03 1.53753630e-01]
 [8.48665536e-01 2.77757870e-01 4.48453530e-04 ... 1.16528704e-01
  7.07399003e-03 1.5375

Case: [' tumor-size', ' irradiat'] with ?
-------------------------------------------
Train-Test sizes:
73% train
27% test
-------------------------------------------
Possible values to predict:
[' tumor-size_0-4', ' tumor-size_10-14', ' tumor-size_15-19', ' tumor-size_20-24', ' tumor-size_25-29', ' tumor-size_30-34', ' tumor-size_35-39', ' tumor-size_40-44', ' tumor-size_45-49', ' tumor-size_5-9', ' tumor-size_50-54', ' tumor-size_?', ' irradiat_?', ' irradiat_no', ' irradiat_yes']
--------------------------------------------
Prediction probability for each entry in test:
[[5.69948973e-03 5.69532222e-03 1.09381354e-02 ... 5.70095709e-02
  9.31012223e-01 1.09080884e-02]
 [9.45041413e-02 1.69031297e-02 5.70232483e-05 ... 9.24340142e-01
  9.16550523e-03 2.78781904e-02]
 [9.45041413e-02 1.69031297e-02 5.70232483e-05 ... 9.24340142e-01
  9.16550523e-03 2.78781904e-02]
 ...
 [8.75508290e-05 2.52634951e-02 4.62372579e-03 ... 8.47641747e-01
  1.88374585e-01 4.05136339e-03]
 [1.92893858e-05 4.

Case: [' node-caps', ' breast'] with ?
-------------------------------------------
Train-Test sizes:
71% train
29% test
-------------------------------------------
Possible values to predict:
[' node-caps_?', ' node-caps_no', ' node-caps_yes', ' breast_?', ' breast_left', ' breast_right', ' breast-quad_?', ' breast-quad_central', ' breast-quad_left_low', ' breast-quad_left_up', ' breast-quad_right_low', ' breast-quad_right_up']
--------------------------------------------
Prediction probability for each entry in test:
[[1.10477164e-02 9.40074292e-01 7.60818100e-03 ... 1.31685398e-02
  5.32341196e-05 7.13910340e-04]
 [6.84091766e-02 7.50541954e-01 3.97714086e-03 ... 2.89386683e-01
  2.39169856e-03 4.09691402e-02]
 [9.02597585e-01 5.71474211e-02 2.15280373e-04 ... 5.84387191e-01
  5.10111144e-05 1.67053793e-03]
 ...
 [9.28275706e-01 5.87718519e-05 1.47609893e-02 ... 2.50158403e-02
  1.26026420e-01 1.84835641e-03]
 [7.83728439e-02 9.38563949e-01 1.26849246e-05 ... 9.62638131e-01
  1.01386

Case: [' breast', ' irradiat'] with ?
-------------------------------------------
Train-Test sizes:
73% train
27% test
-------------------------------------------
Possible values to predict:
[' breast_?', ' breast_left', ' breast_right', ' breast-quad_?', ' breast-quad_central', ' breast-quad_left_low', ' breast-quad_left_up', ' breast-quad_right_low', ' breast-quad_right_up', ' irradiat_?', ' irradiat_no', ' irradiat_yes']
--------------------------------------------
Prediction probability for each entry in test:
[[9.34437470e-01 4.14807566e-02 4.38410720e-02 ... 8.59305975e-03
  9.63831526e-01 2.58880198e-02]
 [1.20468529e-01 3.03246180e-01 1.96583510e-01 ... 5.30175143e-01
  5.30275884e-01 1.26671892e-04]
 [1.20468529e-01 3.03246180e-01 1.96583510e-01 ... 5.30175143e-01
  5.30275884e-01 1.26671892e-04]
 ...
 [6.59814630e-01 2.01139583e-01 8.01309992e-03 ... 9.51533575e-01
  3.74900463e-02 2.96668170e-07]
 [6.59814630e-01 2.01139583e-01 8.01309992e-03 ... 9.51533575e-01
  3.74900463e

# Fare

In [43]:
#percentuale punti interrogativi = # punti interrogativi / numero di righe per numero di colonne